In [2]:
from fastai.vision.all import *
from sklearn.metrics import accuracy_score
from pathlib import Path
from PIL import Image
from PIL import ImageFile
import matplotlib.pyplot as plt

# Allow truncated images 
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Define the paths to the training and testing datasets
train_path = Path('/Users/kylewhite/Downloads/archive-3/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train')
test_path = Path('/Users/kylewhite/Downloads/archive-3/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/test')

# Create DataLoaders for the training dataset
train_dls = ImageDataLoaders.from_folder(
    train_path, 
    valid_pct=0.2,  # 20% of the training data will go to validation
    seed=42,        # Seed for reproducibility
    item_tfms=Resize(128),  # Resize images to 128x128
    bs=64           # Batch size of 64 
)

# Create DataLoader (assuming no labels)
test_dl = train_dls.test_dl(list(test_path.glob('*/*.jpg')) + list(test_path.glob('*/*.png')))

# Create a learner using a pre-trained ResNet18 model
learn = vision_learner(train_dls, resnet18, metrics=accuracy)

# Train the model
learn.fine_tune(2)  # Tune for 2 epochs (accuracy should be around 99.6%)

# Evaluate the model's performance on the validation set
val_preds, val_targets = learn.get_preds(dl=train_dls.valid)
accuracy = accuracy_score(val_targets, val_preds.argmax(dim=1))
print(f"Validation Accuracy: {accuracy:.4f}")

# Save the trained model
learn.export('bone_fracture_classifier.pkl')


epoch,train_loss,valid_loss,accuracy,time


KeyboardInterrupt: 

In [1]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
from fastai.vision.all import load_learner, PILImage

# model path 
MODEL_PATH = '/Users/kylewhite/Downloads/archive-3/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train/bone_fracture_classifier.pkl'

learn = load_learner(MODEL_PATH)

# image classifying function
def classify_image(image_path, learner):
    img = PILImage.create(image_path)
    pred, pred_idx, probs = learner.predict(img)
    return pred, probs[pred_idx]

# window
root = tk.Tk()
root.title("Image Classifier")

# label to display the image
image_label = tk.Label(root, text="No image selected")
image_label.pack(padx=10, pady=10)

# result label
result_label = tk.Label(root, text="")
result_label.pack(padx=10, pady=10)

def upload_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        # display the image
        img = Image.open(file_path)
        img = img.resize((250, 250))  # resize image for display
        img_tk = ImageTk.PhotoImage(img)
        image_label.config(image=img_tk, text="")
        image_label.image = img_tk
        
        # classify image
        pred, prob = classify_image(file_path, learn)
        result_label.config(text=f"Prediction: {pred}\nProbability: {prob:.4f}")

# image button
upload_button = tk.Button(root, text="Upload Image", command=upload_image)
upload_button.pack(padx=10, pady=10)

# run the script
root.mainloop()


2024-09-14 09:13:39.372 Python[71039:5042939] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.
